# Scroll down for content

In [1]:
from IPython.display import HTML
#.css was created using jupyter-themer (grade3 theme) and slightly modified:
# 1) text justification: added 2 lines to .rendered_html * + p block :
#      text-align: justify;
#      text-justify: inter-word;
# 2) fixing first paragraph of block:
#      created a copy of .rendered_html * + p block for a new
#        .rendered_html + p block (no asterisk!) and got rid of top margin
# 3) uniform line spacing of lines with and without inline latex:
#      increased lineheight from 130% to 200%
HTML('<style>'+open('./swap_invariants.css').read()+'</style>')

In [2]:
import warnings
import numpy
import matplotlib.pyplot as plt
import ipywidgets as widgets
from functools import partial

#change default sizes
plt.rcParams['font.size'] = 22
plt.rcParams['figure.figsize'] = (12,10)
#enable latex
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [3]:
to_chi = lambda lmbda: lmbda/(1-lmbda)

In [4]:
# 2d invariants

def two_dimensional():
    n = 2
    D = n
    return (
        lambda x: D-x,
        lambda x: 1/x,
        lambda chi, x: (chi*(D - x) + (D/n)**n)/(chi + x) #(D/n)**n actually reduces to 1
    )
I_sum_2d, I_prod_2d, I_chi_2d = two_dimensional()

def visualize_2d(lmbda, zoom, uniform_axis, fixed_axis, resolution, with_theta):
    fig, ax = plt.subplots()
    chi = to_chi(lmbda)
    
    stop_x = 1 if zoom else fixed_axis if fixed_axis > 0 else I_chi_2d(chi, 0)
    stop_y = fixed_axis if fixed_axis > 0 else I_chi_2d(chi, 0)
    
    x = numpy.linspace(0, stop_x, resolution)
    ax.plot(x, I_sum_2d(x), label=r'$I_\Sigma$',)
    ax.plot(x, I_chi_2d(chi,x), label=r'$I_\chi$',)
    
    eps = 0.00001
    ax.plot([eps]+x[1:], I_prod_2d([eps]+x[1:]), label=r'$I_\Pi$',)
    
    if with_theta:
        theta = lmbda
        phi = numpy.linspace(eps, numpy.pi/2-eps, 2*resolution)
        n = 2
        D = n
        r = theta*(D/(numpy.sin(phi)+numpy.cos(phi))) + (1-theta)*numpy.sqrt(2*D**n/(n**n * numpy.sin(2*phi)))
        ax.plot(r*numpy.cos(phi), r*numpy.sin(phi), label=r'$I_\theta$',)
    
    #naive linear combination
    #tmp1 = lmbda*I_sum_2d(x)
    #tmp1[tmp1 < 0] = 0
    #tmp2 = (1-lmbda)*I_prod_2d(numpy.concatenate([[eps],x[1:]]))
    #print(tmp1[::10])
    #print(tmp2[::10])
    #ax.plot(x, tmp1+tmp2, label='graph linear combination',)

    ax.legend()
    plt.xlabel(r'$x_i$')
    plt.ylabel(r'$x_j$')
    ax.set_xlim(left = 0, right = stop_x)
    ax.set_ylim(bottom = 1 if zoom else 0, top = stop_y)
    plt.title(r'Invariants, $\lambda='+str(lmbda)+r' \Rightarrow \chi='+str(int(chi*100)/100)+
              (r', \theta=\lambda' if with_theta else r'')+r'$')
    if uniform_axis:
        plt.gca().set_aspect('equal', adjustable='box')
    plt.grid(True)
    plt.show()

def interact_2d(with_theta):
    _ = widgets.interact(visualize_2d,\
         lmbda        = widgets.FloatSlider(min=0.01,max=0.99,step=0.01,value=0.5,continuous_update=False),\
         zoom         = widgets.Checkbox(value=False),\
         uniform_axis = widgets.Checkbox(value=False),\
         fixed_axis   = widgets.IntText(value=0,continuous_update=False),\
         resolution   = widgets.IntText(value=100,continuous_update=False),\
         with_theta   = widgets.fixed(with_theta)
    )

In [5]:
# 3d invariants
def three_dimensional():
    n = 3
    D = n
    return (
        lambda x, y: D-x-y,
        lambda x, y: 1/(x*y),
        lambda chi, x, y: (chi*(D - x - y) + (D/n)**n)/(chi + x*y)
    )

I_sum_3d, I_prod_3d, I_chi_3d = three_dimensional()

def visualize_3d(lmbda, zoom, fixed_axis, up_down, left_right, resolution):
    chi = to_chi(lmbda)
    
    stop_x = 1 if zoom else fixed_axis if fixed_axis > 0 else I_chi_3d(chi, 0, 0)
    stop_z = fixed_axis if fixed_axis > 0 else I_chi_3d(chi, 0, 0)
    
    x = numpy.outer(numpy.linspace(0, stop_x, resolution), numpy.ones(resolution))
    y = x.copy().T
    
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore') #technical debt of NaN cut-off handling
        fig = plt.figure()
        ax = plt.axes(projection ='3d')
        fig.set_size_inches(16,16)
        
        def cleanup(z):
            #this is not the proper way to do this - (should probably use numpy masked array)
            # but whatever
            z[z < 0] = numpy.nan
            z[z > stop_z] = numpy.nan
            return z
        
        ax.plot_surface(x, y, cleanup(I_sum_3d(x,y)), label=r'$I_\Sigma$')
        ax.plot_surface(x, y, cleanup(I_chi_3d(chi,x,y)), label=r'$I_\chi$')
        ax.plot_surface(x, y, cleanup(I_prod_3d(x,y)), label=r'$I_\Pi$')
        
        ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$x_3$')
        ax.set_zlim(bottom = 1 if zoom else 0, top = stop_z)
        plt.title(r'Invariants, $\lambda='+str(lmbda)+' \Rightarrow \chi='+str(int(chi*100)/100)+'$')
        
        ax.view_init(up_down, left_right)
        plt.show()

def interact_3d():
    widgets.interact(visualize_3d,\
        lmbda      = widgets.FloatSlider(min=0.01,max=1,step=0.01,value=0.5,continuous_update=False),\
        zoom       = widgets.Checkbox(value=False, description='zoom'),\
        fixed_axis = widgets.IntText(value=2,continuous_update=False),\
        up_down    = widgets.IntSlider(min=0,max=90,step=2,value=20,continuous_update=False),\
        left_right = widgets.IntSlider(min=-90,max=90,step=2,value=0,continuous_update=False),\
        resolution = widgets.IntText(value=50,continuous_update=False),\
    )

In [6]:
# amp decay
def amp_decay(n):
    resolution = 20
    x = numpy.linspace(1, resolution, resolution)
    #n different tokens, each with a balance of 1 in equilibrium, split into x pieces for nx pieces in total
    #first token has balance of 1 piece -> rho_1 = 1/x
    #nx-1 pieces remaining split evenly over remaining n-1 tokens hence
    #rho_i = (nx-1)/(n-1)/x
    rho_1 = 1/x
    rho_i = (n*x - 1)/(x*(n-1))
    decay = rho_1 * rho_i**(n-1)
    fig, ax = plt.subplots()
    
    ax.bar(x, decay, label='Amp Decay',)
    plt.xlabel(r'$\frac{1}{\rho}$')
    plt.ylabel('Amp Factor Scaling')
    plt.xticks(numpy.arange(1,resolution+1,1))
    plt.yticks(numpy.arange(0,1.1,0.1))
    plt.title('Curve Amp Decay (Lower Bound)')
    plt.grid(True)
    plt.show()

def interact_decay():
    widgets.interact(amp_decay,\
        n = widgets.IntSlider(min=2,max=6,step=1,value=2,continuous_update=False),\
    )

## Basic Invariants

Assume a pool with $n$ different tokens with balances $x_i, i = 1 \ldots n$.

First, consider the constant **sum** formula (where $D$ can be considered the depth of the pool):
$$ I_\Sigma := \left[ \sum_{i=1}^{n} x_i = D \right] $$

If we assume that there is an equal number of all tokens in the pool, i.e. $\forall i,j: x_i = x_j$ (we also call such a pool in equilibrium) then $x_i = \frac{D}{n}$ and hence if we were to apply the constant **product** formula instead, we'd get:
$$ I_\Pi := \left[ \prod_{i=1}^{n} x_i = \left( \frac{D}{n} \right)^n \right] $$

Now that we have two invariants $I_\Sigma$ and $I_\Pi$, any [convex combination](https://en.wikipedia.org/wiki/Convex_combination) of the two will yield a new invariant that lies between the two:

$$ I_\lambda := \lambda I_\Sigma + (1 - \lambda) I_\Pi \quad , \lambda \in (0,1) $$

alternatively (to use the notation of the original [curve whitepaper](https://curve.fi/files/stableswap-paper.pdf)), we can divide both sides of $I_\lambda$ by $1-\lambda$ and define $\chi = \frac{\lambda}{1-\lambda}$ to get an alternative form:

$$ I_\chi := \chi I_\Sigma + I_\Pi \quad , \chi \in (0, \infty) $$

plugging in, we get:
$$ I_\chi = \left[ \chi \sum_{i=1}^{n} x_i + \prod_{i=1}^{n} x_i = \chi D + \left( \frac{D}{n} \right)^n \right] $$

## Visualization

To get an idea what this invariant looks like, let's plot it for a pool with $n = 2$ and $n = 3$ tokens respectively.

To that end, we need to express $x_j$ as a function of all other $x_i, i \neq j$, $D$, and $\chi$. With a bit of rearranging we get:
$$ x_j = \frac{\chi \left( D - \sum_{i \neq j} x_i \right) + \left( \frac{D}{n} \right)^n}{\chi + \prod_{i \neq j} x_i} $$

Without loss of generality, we can set $D = n$ (since the unit of each token is arbitrary and hence we can always think of them as being normalized to 1 which then gives rise to $\sum x_i = D = n$).

In [7]:
interact_2d(False)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='lmbda', max=0.99, min=0.01,…

In [8]:
interact_3d()

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='lmbda', max=1.0, min=0.01, …

So, as expected, we see that $I_\chi$ is sandwiched between $I_\Sigma$ and $I_\Pi$. One feature that might perhaps seem surprising is that unlike $I_\Pi$, $I_\chi$ does not diverge when any of the $x_i$ approaches 0. The intuition that's leading one astray if one has that expectation is that a convex combination of the invariants is neither the cartesian nor the polar convex combination of the graphs representing them.

## Radial Distance Invariant

In fact, let's express $I_\Sigma$ and $I_\Pi$ in polar cooridnates for the 2-dimensional case (i.e. $x_1 = r \cos \phi, x_2 = r \sin \phi$):

$$ \text{reminder:} \quad \sin \alpha + \cos \alpha = \sqrt{2} \sin \left( \alpha + \frac{\pi}{4} \right), \quad \sin \alpha \cos \alpha = \frac{\sin 2\alpha}{2} $$

$$ I_\Sigma = \left[ r(\cos \phi + \sin \phi) = D \right] \quad \text{ hence } \quad r = \frac{D}{\sin \phi + \cos \phi}, \quad \phi = \arcsin \left( \frac{D}{\sqrt{2}r} \right) - \frac{\pi}{4} $$

$$ I_\Pi = \left[ r^2 \cos \phi \sin \phi = \left( \frac{D}{2} \right)^2 \right] \quad \text{ hence } \quad r = \frac{D}{2} \sqrt{\frac{2}{\sin 2 \phi}}, \quad \phi = \frac{1}{2} \arcsin \left( \frac{D^2}{2 r^2} \right) $$

Armed with these formulae, we can consider yet another invariant $I_\theta, \theta \in (0,1)$ that uses an convex combination of the radial distances of the corresponding points of $I_\Sigma$ and $I_\Pi$ for a given polar angle $\phi$ (i.e. shoot a ray from the origin at angle $\phi$, find the points of intersection with $I_\Sigma, I_\Pi$ and call them $p_\Sigma, p_\Pi$, to find the correspoinding point $p_\theta = \theta p_\Sigma + (1-\theta) p_\Pi$ of $I_\theta$ that also lies on the ray at a relative distance of $\theta$ between the two). So in 2 dimensions we get:

$$ I_\theta^{(2d)} = \left[ r = \theta \left( \frac{D}{\sin \phi + \cos \phi} \right) + (1 - \theta) \left( \frac{D}{2} \sqrt{\frac{2}{\sin 2 \phi}} \right) \right] $$

Visualizing it along with the other variants:

In [9]:
interact_2d(True)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='lmbda', max=0.99, min=0.01,…

Now this approach could be extended to higher dimension (i.e. more coins) and would perhaps benefit from a formulation using projective geometry (which might allow us to transform all those pesky trigonometric functions into linear algebra instead).

For now however, we'll instead turn to the last step in the Curve whitepaper and reconstruct their stableswap invariant to see how it compares to the invariants we have so far.

## Curve Stableswap Invariant

The Curve whitepaper constructs $I_\chi$, observes, just as we did, that it intersects the axes (in their words: "However, it wouldn't support prices going far from the ideal price 1.0.") but then takes a different route to achieve the desired divergence along the axes than we have with our Radial Distance Invariant $I_\theta$ by making $\chi$ vary based on the token balances in the pool (i.e. turning $\chi$ into a function $\chi(\overrightarrow{x}), \overrightarrow{x} = (x_1, \ldots, x_n)$) which can take any positive value but tends towards 0 as the pool disequilibrates, hence pushing the overall invariant towards the constant product invariant $I_\Pi$:

$$ \chi(\overrightarrow{x}) = Amp \cdot Decay(\overrightarrow{x}) $$

$Amp$ is the amplification factor which takes on the role of leverage (with larger $Amp$ pushing the pool invariant towards $I_\Sigma$ and hence reducing slippage on swaps).

To understand $Decay$ we first consider the ratio $\rho_i = \frac{x_i}{D/n}$. $\rho_i$ can be thought of as a normalized version of $x_i$ that specifies the token's relative abundance in the pool. A pool in equilibrium, yields $\rho_i = 1$ for all its tokens, but as the pool disequilibrates, some token balances have to decrease while others increase and hence some $\rho_i$ will have to go down as other $\rho_j$ rise.

Now, if we were to use the constant product formula, $\rho_j$ would rise exactly enough so that the product of $\rho_i$ and $\rho_j$ remains constant. For example, in a pool with $x_1 = 2, x_2 = 2$ (and hence $\rho_1 = \rho_2 = 1$) one could buy one token of the first type for two tokens of the second type yielding $x_1 = 1, x_2 = 4$ (and hence $\rho_1 = \frac{1}{2}, \rho_2 = 2$). If we consider the same pool but consider the constant sum formula however, we could trade 1 token of the first kind for 1 token of the second kind yielding token balances of $x_1 = 1, x_2 = 3$ and consequently $\rho_1 = \frac{1}{2}, \rho_2 = \frac{3}{2}$ for a product of $\frac{3}{4} < 1$.

So, since $I_\chi$ always lies somewhere between $I_\Sigma$ and $I_\Pi$, $\prod_i \rho_i$ starts off at $1$ when the pool is in equilibrium, decreases monotonically as token balances drift out of balance, and tends towards 0 as the relative abundance of even one token tends towards 0. Therefore $\prod_i \rho_i$ is a valid choice for $Decay(\overrightarrow{x})$ and in fact the one used by Curve:

$$ \chi(\overrightarrow{x}) = Amp \prod_{i=1}^n \frac{x_i}{D/n} = Amp \frac{\prod_i x_i}{(D/n)^n} $$

The following graph visualizes $Decay$ for a pool with $n$ tokens where one token's $\rho_i = 1/x$ while the surplus is spread evenly among the other tokens. In fact, the following graph constitutes a lower bound because it actually uses a constant sum invariant, so with the actual stableswap invariant the decay would be even slower (i.e. longer bars):

In [10]:
interact_decay()

interactive(children=(IntSlider(value=2, continuous_update=False, description='n', max=6, min=2), Output()), _…

Notice that there are plenty of other, readily available candidates for $Decay$ such as $\min_i(\rho_i)$ and that if $f$ is a valid choice for $Decay$, so is $f^s, s > 0$.

Finally, using $Amp = A \cdot D^{n-1}$ (with $A$ being a parameter), plugging $\chi(\overrightarrow{x})$ into $I_\chi$, and diving by $\prod_i x_i$ we get the Curve/Stableswap invariant $I_C(A)$:

$$ I_C(A) := \left[ \frac{A n^n}{D} \sum_{i=1}^{n} x_i + 1 = A n^n + \underbrace{\frac{(D/n)^n}{\prod_{i=1}^n x_i}}_{=Decay^{-1}} \right] $$

### Stableswap Numerics

Since $I_C$ has no explicit form for $x_j$ and $D$ we'll have to turn to iterative methods instead.

Our approach will be: Putting $I_C$ into its [implicit form](https://en.wikipedia.org/wiki/Implicit_function) $F(z) = 0$ (where $z$ is either some $x_j$ or $D$) and then employing [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method) by iterating $G(z) = z - \frac{F(z)}{F'(z)}$ until the desired precision is reached.

#### Finding $x_j$

\begin{align*}
F(x_j)  & = A n^n + \frac{(D/n)^n}{\prod_i x_i} - \frac{A n^n}{D} \sum_i x_i - 1 \\
F'(x_j) & = - \frac{1}{x_j} \frac{(D/n)^n}{\prod_i x_i} - \frac{A n^n}{D} \\
        & \stackrel{(1)}{=} - \frac{1}{x_j} \left( \frac{A n^n}{D} \sum_{i=1}^{n} x_i + 1 - A n^n \right) - \frac{A n^n}{D}
\end{align*}

\begin{align*}
(1) & \text{ use invariant to substitute for } \frac{(D/n)^n}{\prod_i x_i}
\end{align*}

Now, using Newton's method:

\begin{align*}
G(x_j) & = x_j - \frac{F(x_j)}{F'(x_j)} \\
       & \stackrel{(1)}{=} \frac{\left( \frac{A n^n}{D} \sum_{i=1}^{n} x_i + 1 - A n^n + \frac{A n^n}{D} x_j \right) + \left( A n^n + \frac{(D/n)^n}{\prod_i x_i} - \frac{A n^n}{D} \sum_i x_i - 1 \right)}{\frac{1}{x_j} \left( \frac{A n^n}{D} \sum_{i=1}^{n} x_i + 1 - A n^n \right) + \frac{A n^n}{D}} \\
       & \stackrel{(2)}{=} \frac{x_j \left( \frac{A n^n}{D} x_j + \frac{(D/n)^n}{\prod_i x_i} \right)}{\frac{A n^n}{D} \sum_{i=1}^{n} x_i + 1 - A n^n + \frac{A n^n}{D} x_j} \\ 
       & \stackrel{(3)}{=} \frac{x_j^2 + \frac{D}{A n^n} \frac{(D/n)^n}{\prod_{i \neq j} x_i}}{\sum_{i=1}^{n} x_i + \frac{D}{A n^n} - D + x_j} \\
\end{align*}

\begin{align*}
(1) & \text{ substitute, absorb minus sign into denominator, and convert to same denominator} \\
(2) & \text{ simplify numerator and expand by } x_j \\
(3) & \text{ expand by } D/(A n^n) \\
\end{align*}

We verify with Curve's [3pool](https://github.com/curvefi/curve-contract/blob/b0bbf77f8f93c9c5f4e415bce9cd71f0cdee960e/contracts/pools/3pool/StableSwap3Pool.vy#L356) code and find that things check out:
\begin{align*}
  y & = x_j \\
  c & = \frac{D}{A n^n} \frac{(D/n)^n}{\prod_{i \neq j} x_i} \\
  b & = \sum_{i \neq j} x_i + \frac{D}{A n^n} \\
  y_{\text{next}} & = \frac{y^2 + c}{2 y + b - D} \\
    & = \frac{x_j^2 + \frac{D}{A n^n} \frac{(D/n)^n}{\prod_{i \neq j} x_i}}{2 x_j + \sum_{i \neq j} x_i + \frac{D}{A n^n} - D}
\end{align*}

#### Finding $D$

\begin{align*}
F(D)  & = A n^n D + D \frac{(D/n)^n}{\prod_{i=1}^n x_i} - A n^n \sum_{i=1}^{n} x_i - D \\
F'(D) & = A n^n + (n+1) \frac{(D/n)^n}{\prod_{i=1}^n x_i} - 1
\end{align*}

Again, using Newton's method:

\begin{align*}
G(D) & = D - \frac{F(D)}{F'(D)} \\
     & = \frac{\left( A n^n D + (n+1) D \frac{(D/n)^n}{\prod_i x_i} - D \right) - \left( A n^n D + D \frac{(D/n)^n}{\prod_i x_i} - A n^n \sum_i x_i - D \right)}{A n^n + (n+1) \frac{(D/n)^n}{\prod_i x_i} - 1} \\
     & = \frac{n D \frac{(D/n)^n}{\prod_i x_i} + A n^n \sum_i x_i}{A n^n + (n+1) \frac{(D/n)^n}{\prod_i x_i} - 1}
\end{align*}

Using $D_0 = \sum_i x_i$ for a pool in equilibrium, we can then iterate $D_{m+1} = G(D_m)$ until it converges to $G$'s fixed point $D$ (which is exactly what's implemented in Curve's [3pool](https://github.com/curvefi/curve-contract/blob/b0bbf77f8f93c9c5f4e415bce9cd71f0cdee960e/contracts/pools/3pool/StableSwap3Pool.vy#L195) ).